In [89]:
from math import sqrt
from itertools import permutations

# Iterators and Generators

## Lecture

### Generators

In [32]:
def f():
    yield 2

type(f), type(f())

(function, generator)

Generator function contains one or more yield statements.

It returns an iterator but does not start execution immediately 

Methods like `__iter__()` and `__next__()` are implemented automatically

In [34]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        index = 0
        while index < self.end:
            yield index
            index += 1


x = CustomRange(1, 3)

type(x), type(x.__iter__), type(iter(x))

(__main__.CustomRange, method, generator)

In [39]:
def my_gen():
    n = 1
    print('This is printed first')
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n

type(my_gen), type(my_gen())

(function, generator)

In [40]:
my_gen()

<generator object my_gen at 0x000001E320FF77B0>

In [41]:
for i in my_gen():
    pass

This is printed first
This is printed second
This is printed at last


In [43]:
for i in my_gen():
    print(i)

This is printed first
1
This is printed second
2
This is printed at last
3


### Generator expression

In [46]:
print((x**2 for x in [1, 2]))

<generator object <genexpr> at 0x000001E32101C5F0>


In [49]:
print([x**2 for x in [1, 2]])

[1, 4]


In [48]:
def g():
    for x in [1, 2]:
        yield x**2
        
print(g())

<generator object g at 0x000001E32101C660>


## Lab

### Custom range

#### Instance as iterator

In [6]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.value > self.end:
            raise StopIteration
        
        current_value = self.value
        self.value += 1
        return current_value


x = CustomRange(1, 3)

iter1 = iter(x)
iter2 = iter(x)

for n in iter1:
    print(n)

for n in iter2:
    print(n)

1
2
3


In [7]:
x = CustomRange(1, 3)

for n in x:
    print(n)
    
for n in x:
    print(n)

1
2
3


#### External iterator class

In [8]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        return CustomRangeIterator(self)


class CustomRangeIterator:
    def __init__(self, custom_range_obj):
        self.custom_range_obj = custom_range_obj
        self.value = custom_range_obj.start
        
    def __iter__(self):
        return self    
        
    def __next__(self):
        if self.value > self.custom_range_obj.end:
            raise StopIteration
        
        current_value = self.value
        self.value += 1
        return current_value


x = CustomRange(1, 3)

iter1 = iter(x)
iter2 = iter(x)

for n in iter1:
    print(n)

for n in iter2:
    print(n)

1
2
3
1
2
3


In [9]:
x = CustomRange(1, 3)

for n in x:
    print(n)

for n in x:
    print(n)

1
2
3
1
2
3


#### Internal iterator class

In [11]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        return self.Iterator(self)

    class Iterator:
        def __init__(self, custom_range_obj):
            self.custom_range_obj = custom_range_obj
            self.value = custom_range_obj.start

        def __iter__(self):
            return self    

        def __next__(self):
            if self.value > self.custom_range_obj.end:
                raise StopIteration

            current_value = self.value
            self.value += 1
            return current_value


x = CustomRange(1, 3)

iter1 = iter(x)
iter2 = iter(x)

for n in iter1:
    print(n)

for n in iter2:
    print(n)

1
2
3
1
2
3


In [12]:
x = CustomRange(1, 3)

for n in x:
    print(n)

for n in x:
    print(n)

1
2
3
1
2
3


#### Reversed iterator

In [17]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        return self.Iterator(self)
    
    def __reversed__(self):
        return self.ReverseIterator(self)

    class Iterator:
        def __init__(self, custom_range_obj):
            self.custom_range_obj = custom_range_obj
            self.value = custom_range_obj.start

        def __iter__(self):
            return self    

        def __next__(self):
            if self.value > self.custom_range_obj.end:
                raise StopIteration

            current_value = self.value
            self.value += 1
            return current_value
        
    class ReverseIterator:
        def __init__(self, custom_range_obj):
            self.custom_range_obj = custom_range_obj
            self.value = custom_range_obj.end

        def __iter__(self):
            return self    

        def __next__(self):
            if self.value < self.custom_range_obj.start:
                raise StopIteration

            current_value = self.value
            self.value -= 1
            return current_value



cr = CustomRange(1, 3)

for x in cr:
    print(f'Iter: {x}')

for x in reversed(cr):
    print(f'Reversed: {x}')

Iter: 1
Iter: 2
Iter: 3
Reversed: 3
Reversed: 2
Reversed: 1


#### Combined iterator

In [18]:
class CustomRange:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.value = start
        
    def __iter__(self):
        return self.Iterator(self, is_reversed = False)
    
    def __reversed__(self):
        return self.Iterator(self, is_reversed = True)

    class Iterator:
        def __init__(self, custom_range_obj, is_reversed):
            self.custom_range_obj = custom_range_obj
            self.is_reversed = is_reversed
            if is_reversed:
                self.value = custom_range_obj.end
            else:
                self.value = custom_range_obj.start
            

        def __iter__(self):
            return self   

        def __next__(self):
            if self.value > self.custom_range_obj.end or self.value < self.custom_range_obj.start:
                raise StopIteration

            current_value = self.value
            if self.is_reversed:
                self.value -= 1
            else:
                self.value += 1
            return current_value
   

cr = CustomRange(1, 3)

for x in cr:
    print(f'Iter: {x}')

for x in reversed(cr):
    print(f'Reversed: {x}')

Iter: 1
Iter: 2
Iter: 3
Reversed: 3
Reversed: 2
Reversed: 1


### Vowels

#### Iterator is the instance

In [25]:
class vowels:
    def __init__(self, text):
        self.text = text
    
    def __iter__(self):
        self.position_in_text = 0
        return self
    
    def __next__(self):
        if self.position_in_text >= len(self.text):
            raise StopIteration
        
        char = self.text[self.position_in_text]
        self.position_in_text += 1
        
        if not self.is_vowel(char):
            return self.__next__()

        return char
        
    @staticmethod
    def is_vowel(char):
        return char.lower() in {"a", "e", "o", "u", "i"}
    

my_string = vowels('Abcedifuty0o')
for char in my_string:
    print(char)

A
e
i
u
o


#### Generator function

In [28]:
VOWELS = {"a", "e", "o", "u", "i"}
def vowel(text):
    for ch in text:
        if ch.lower() in VOWELS:
            yield ch

text = "ajsdijqp12i3-01 mcpozmc,19i3= 910ezxczxcvkp[eqitgo]"
generator = vowel(text)

print(next(generator))
print(next(generator))
print(next(generator))

a
i
i


#### Generator comprehension

In [29]:
def vowel(text):
    return (ch for ch in text if ch.lower() in VOWELS)

text = "ajsdijqp12i3-01 mcpozmc,19i3= 910ezxczxcvkp[eqitgo]"
generator = vowel(text)

print(next(generator))
print(next(generator))
print(next(generator))

a
i
i


### Squares

In [56]:
def squares(n):
    for i in range(1, n + 1):
        yield i ** 2
        
x = squares(3)

for i in x:
    print(i)
    
print(squares(2))

1
4
9
<generator object squares at 0x000001E32101CF20>


### Reverse text

In [58]:
def reverse_text(text):
    for ch in text[::-1]:
        yield ch

for char in reverse_text("step"):
    print(char)

p
e
t
s


## Other

### Fibonacci

In [78]:
def fibonacci():
    current = 0
    following = 1
    
    while True:
        yield current
        current, following = following, current + following

generator = fibonacci()
for _ in range(5):
    print(next(generator))

0
1
1
2
3


### Possible permutations

In [79]:
def generate_permutations(ll):
    return (p for p in permutations(ll))

permutation_generator = generate_permutations([1, 2, 3])

for p in permutation_generator:
    print(p)   

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [80]:
x = permutations(list('asd'))
list(x)

[('a', 's', 'd'),
 ('a', 'd', 's'),
 ('s', 'a', 'd'),
 ('s', 'd', 'a'),
 ('d', 'a', 's'),
 ('d', 's', 'a')]

### Prime number

In [81]:
def is_prime(number):
    if number in (0, 1):
        return False

    for x in range(2, int(sqrt(number)) + 1):
        if number % x == 0:
            return False

    return True


def get_primes(integer_list):
    for number in integer_list:
        if is_prime(number):
            yield number


print(list(get_primes([2, 4, 3, 5, 6, 9, 1, 0])))

[2, 3, 5]


### Primes generator

In [84]:
def is_prime(number):
    for x in range(2, int(sqrt(number)) + 1):
        if number % x == 0:
            return False

    return True


def generate_primes(max_number):
    number = 1
    while number <= max_number:
        if is_prime(number):
            yield number
        number += 1

        
primes = generate_primes(100)
print(next(primes))
print(next(primes))
print(next(primes))
print(next(primes))

1
2
3
5


### Take skip

In [88]:
class take_skip:
    def __init__(self, step, count):
        self.step = step
        self.count = count

    def __iter__(self):
        return (n * self.step for n in range(self.count))



numbers = take_skip(1.5, 4)
for number in numbers:
    print(number)

0.0
1.5
3.0
4.5
